In this post, we compute symbolic expressions of the apparent stiffness introduced in [this post]({filename}202105XX-What_is_homogenization-06.md). Our goal is to find the solution to the general problem depicted below.

![The problem considered here]({static}What_is_homogenization/uniaxial_tension.png){.fig60p100}

## Outline of the derivation

We consider a simplified case, where the mesh is square and vertical and horizontal springs have equal stiffnesses. We use the same symbols an in the [previous post]({filename}). In particular, the stiffness of the diagonal springs is $k$, while the stiffness of the horizontal and vertical springs is $\chi k$ ($\chi$: dimensionless parameter).

To compute the solution to this problem, we use a displacement-based approach relying on the minimization of the total potential energy, $\Pi$. This energy is the difference between the strain energy, $\mathcal U$ and the potential of external forces, $\mathcal V$. Both $\mathcal U$ and $\mathcal V$ are functions of the nodal displacements, that must satisfy the essential boundary conditions. The numerous symmetries will also allow us to reduce the number of unknowns.

In a previous post, we derived [the stiffness matrix of a linear spring]({filename}20201125-On_the_stiffness_matrix_of_a_linear_spring.md). This expression is used in the present post to evaluate the strain energy $\mathcal U$

$$\mathcal U=\sum_{i}\tfrac12k_i\bigl(\Delta\vec u_i\cdot\vec n_i\bigr)^2,$$

where the sum runs over all springs (indexed by $i$), $k_i$ is the stiffness of the spring, $\vec n_i$ its direction (unit vector) and $\Delta u_i$ is the relative displacement of its two end-points.

The potential of external forces is the following sum

$$\mathcal V=\sum_i\vec Q_i\cdot\vec u_i,$$

where the sum runs over all nodes that are loaded, $\vec Q_i$ is the applied nodal force and $\vec u_i$ is the nodal displacement.

Minimization of the potential energy $\Pi=\mathcal U-\mathcal V$ with respect to the unknown nodal displacements delivers the solution.

The derivation is carried out with the [Sympy](https://www.sympy.org) library, that we first import. Note that we will also use the [NumPy](https://numpy.org/) library to build arrays of Sympy expressions and use vectorized operations.

In [ ]:
import numpy as np
import sympy

We define a few common symbols, to be used in all subsequent derivations.

The geometry of the grid is defined by the angle between the horizontal and diagonal springs.

In [ ]:
θ = sympy.pi / 4

We then define the intensity of the nodal forces

In [ ]:
Q = sympy.Symbol("Q")

the stiffness of the diagonal springs

In [ ]:
k = sympy.Symbol("k")

the relative stiffness of the horizontal and vertical springs

In [ ]:
χ = sympy.Symbol("chi")
χx = χ
χy = χ

and the stiffness of the horizontal and vertical springs

In [ ]:
kx = χx * k
ky = χy * k

We also define the unit vectors that give the directions of each spring.

In [ ]:
one = sympy.Number(1)
zero = sympy.Number(0)
e1 = np.array([one, zero])
e2 = np.array([zero, one])
d1 = np.array([sympy.cos(θ), sympy.sin(θ)])
d2 = np.array([-sympy.cos(θ), sympy.sin(θ)])

And we are ready to proceed!

## Definition of some common functions

In [ ]:
def strain_energy(u):
    N = u.shape[0] - 1
    U = zero
    for x in range(N + 1):
        for y in range(N + 1):
            # Horizontal springs
            if x < N:
                U += kx / 2 * (e1.dot(u[x + 1, y] - u[x, y])) ** 2
            # Vertical springs
            if y < N:
                U += ky / 2 * (e2.dot(u[x, y + 1] - u[x, y])) ** 2
            if (x < N) and (y < N):
                U += k / 2 * (d1.dot(u[x + 1, y + 1] - u[x, y])) ** 2
                U += k / 2 * (d2.dot(u[x, y + 1] - u[x + 1, y])) ** 2
    return U

In [ ]:
def potential_external_forces(u):
    N = u.shape[0] - 1
    V = (u[-1, 0, 0] + u[-1, -1, 0] - u[0, 0, 0] - u[0, -1, 0]) / 2
    for y in range(1, N):
        V += u[-1, y, 0] - u[0, y, 0]
    V *= F / N
    return V

In [ ]:
def potential_energy(u):
    U = strain_energy(u)
    V = potential_external_forces(u)
    return U - V

## The 1×1 case

```
B     A
 •───•
 │╲ ╱│
 │ ╳ │
 │╱ ╲│
 •───•
B'   A'
```

In [ ]:
uA, vA = dofs = sympy.symbols("u_A, v_A")

In [ ]:
N = 1
u = np.empty((N + 1, N + 1, 2), dtype=object)
u[0, 0] = -uA, -vA
u[1, 0] = uA, -vA
u[0, 1] = -uA, vA
u[1, 1] = uA, vA

In [ ]:
U1 = sympy.expand(strain_energy(u))
V1 = sympy.expand(potential_external_forces(u))
Π1 = U1 - V1
Π1

In [ ]:
eqs = [Π1.diff(dof) for dof in dofs]
sol = sympy.solve(eqs, dofs)
for key, value in sol.items():
    display(sympy.Eq(key, value))

In [ ]:
ΔL = V1.subs(F, one).subs(sol)
K1 = F / ΔL
K1_red = sympy.factor(K1 / k)
K1_red

## The 2×2 case

In [ ]:
uA, vA, vB, uC = dofs = sympy.symbols("u_A v_A v_B u_C")
u = np.empty((3, 3, 2), dtype=object)
u[0, 0] = -uA, -vA
u[1, 0] = zero, -vB
u[2, 0] = uA, -vA
u[0, 1] = -uC, zero
u[1, 1] = zero, zero
u[2, 1] = uC, zero
u[0, 2] = -uA, vA
u[1, 2] = zero, vB
u[2, 2] = uA, vA

In [ ]:
U2 = sympy.expand(strain_energy(u))
V2 = sympy.expand(potential_external_forces(u))
Π2 = U2 - V2
Π2

In [ ]:
eqs = [Π2.diff(dof) for dof in dofs]
sol = sympy.solve(eqs, dofs)
for key, value in sol.items():
    display(sympy.Eq(key, value.factor()))

In [ ]:
ΔL = V2.subs(F, one).subs(sol)
K2 = sympy.factor(F / ΔL)
K2_red = K2 / k
K2_red

## The 3×3 case

In [ ]:
uA, vA, uB, vB, uC, vC, uD, vD = dofs = sympy.symbols("u_A v_A u_B v_B u_C v_C u_D v_D")
u = np.empty((4, 4, 2), dtype=object)

u[0, 0] = -uA, -vA
u[1, 0] = -uB, -vB
u[2, 0] = uB, -vB
u[3, 0] = uA, -vA

u[0, 1] = -uC, -vC
u[1, 1] = -uD, -vD
u[2, 1] = uD, -vD
u[3, 1] = uC, -vC

u[0, 2] = -uC, vC
u[1, 2] = -uD, vD
u[2, 2] = uD, vD
u[3, 2] = uC, vC

u[0, 3] = -uA, vA
u[1, 3] = -uB, vB
u[2, 3] = uB, vB
u[3, 3] = uA, vA

In [ ]:
U3 = sympy.expand(strain_energy(u))
V3 = sympy.expand(potential_external_forces(u))
Π3 = U3 - V3
Π3

In [ ]:
eqs = [Π3.diff(dof) for dof in dofs]
sol = sympy.solve(eqs, dofs)
for key, value in sol.items():
    display(sympy.Eq(key, value))

In [ ]:
ΔL = V3.subs(F, one).subs(sol)
K3 = sympy.factor(F / ΔL)
K3_red = K3 / k
K3_red